### **Problema de clasificación de audio**

El problema trata de clasificar un audio intentando predecir a qué clase pertenece. Las distintas clases se dividen por la intención que tenga el usuario al hacer la consulta. En nuestro caso tenemos: `efectos_adversos` y `otros`.

In [2]:
import os
import pandas as pd

#### 1. **Generamos dataframe con audios y clases**

In [14]:
# 1) Carpeta base donde están tus audios organizados por subcarpeta = etiqueta
AUDIO_BASE = "data/outputs/6_audio"
# 2) Ruta donde guardar el CSV (si quieres)
OUT_CSV = "data/metadata.csv"

# 3) Recorre las subcarpetas y recoge filename + label
rows = []
for label in os.listdir(AUDIO_BASE):
    class_dir = os.path.join(AUDIO_BASE, label)
    if os.path.isdir(class_dir):
        for fname in os.listdir(class_dir):
            if fname.lower().endswith(".wav"):
                rows.append({
                    "filename": fname,
                    "label": label
                })

# 4) Crea el DataFrame
df = pd.DataFrame(rows, columns=["filename", "label"])

# 5) (Opcional) Guárdalo en CSV
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
df.to_csv(OUT_CSV, index=False)

# 6) Comprueba resultado
print(f"DataFrame con {len(df)} filas y {len(df.columns)} columnas:")
print(df.head())

DataFrame con 20 filas y 2 columnas:
             filename             label
0  audio_10_angel.wav  efectos_adversos
1   audio_1_angel.wav  efectos_adversos
2   audio_2_angel.wav  efectos_adversos
3   audio_3_angel.wav  efectos_adversos
4   audio_4_angel.wav  efectos_adversos


#### 2. **Clasificación de audio**

In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection  import train_test_split, GridSearchCV
from sklearn.preprocessing   import StandardScaler
from sklearn.ensemble        import RandomForestClassifier
from sklearn.pipeline        import Pipeline
from sklearn.metrics         import classification_report, confusion_matrix

from utils_audio import (
    preprocess_audio,
    extract_zcr,
    extract_centroid_rolloff,
    extract_mfcc
)

# 1) Cargamos el DataFrame
df = pd.read_csv("data/metadata.csv")
AUDIO_BASE = "data/outputs/6_audio"

# 2) Función para vectorizar un clip (igual que antes)
def extract_clip_features(label: str, fname: str):
    clip_path = os.path.join(AUDIO_BASE, label, fname)
    y = preprocess_audio(clip_path, reduce_noise_flag=False)
    if y is None:
        print(f"Skipping (preproc failure): {clip_path}")
        return None

    zcr        = extract_zcr(y, sr=16000)                   
    centroid, rolloff = extract_centroid_rolloff(y, sr=16000)  
    mfcc       = extract_mfcc(y, sr=16000, n_mfcc=13)        

    feats = []
    for arr in (zcr, centroid, rolloff):
        feats += [arr.mean(), arr.std()]
    feats += list(mfcc.mean(axis=1)) + list(mfcc.std(axis=1))

    return np.array(feats)

# 3) Recorremos df para llenar X_list, y_list
X_list, y_list = [], []
for _, row in df.iterrows():
    feats = extract_clip_features(row['label'], row['filename'])
    if feats is not None:
        X_list.append(feats)
        y_list.append(row['label'])

# Ahora sí construimos X e y
X = np.vstack(X_list)      # (n_clips, n_features)
y = np.array(y_list)       # (n_clips,)

# 4) Split, pipeline, entrenamiento…
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(random_state=42))
])

param_grid = {
    "clf__n_estimators": [100, 200],
    "clf__max_depth":    [None, 10, 20],
}

grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

print("Mejores parámetros:", grid.best_params_)

y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))


TypeError: argument of type 'NoneType' is not iterable